In [1]:
%%capture
!pip install trl

In [2]:
import os
import torch
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
os.environ["WANDB_DISABLED"] = "true"

In [3]:
class LengthSampler:
    def __init__(self, min_value, max_value):
        self.values = list(range(min_value, max_value))
    def __call__(self):
        return np.random.choice(self.values)

input_size = LengthSampler(2, 8)
output_size = LengthSampler(4, 16)

In [4]:
generative_model = "lvwerra/gpt2-imdb"
active_model = AutoModelForCausalLMWithValueHead.from_pretrained(generative_model)
reference_model = AutoModelForCausalLMWithValueHead.from_pretrained(generative_model)

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(generative_model)
tokenizer.pad_token = tokenizer.eos_token
def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["review"])[:input_size()]
    sample["query"] = tokenizer.decode(sample["input_ids"])
    return sample
dataset = load_dataset("imdb", split="train")
dataset = dataset.select(range(min(len(dataset), 500)))
dataset = dataset.rename_columns({"text": "review"})
dataset = dataset.filter(lambda x: len(x["review"]) > 200, batched=False)
dataset = dataset.map(tokenize, batched=False)
dataset.set_format(type="torch")

tokenizer_config.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/495 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


In [6]:
config = PPOConfig(model_name=generative_model, learning_rate=1.41e-5)
ppo_trainer = PPOTrainer(config, active_model, reference_model, tokenizer, dataset=dataset, data_collator=lambda data: {key: [d[key] for d in data] for key in data[0]})
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


In [7]:
sentiment_model = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
sentiment_kwargs = {"top_k": None, "function_to_apply": "none", "batch_size": 16}
generation_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    response_tensors = []
    for query in query_tensors:
        generation_kwargs["max_new_tokens"] = output_size()
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-output_size():])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    sentiment_outputs = sentiment_model(texts, **sentiment_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in sentiment_outputs]
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

0it [00:00, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
3it [01:05, 21.79s/it]


In [9]:
query = "This movie is"
query_tensor = tokenizer.encode(query, return_tensors="pt").squeeze()
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}
reference_output = reference_model.generate(query_tensor.clone().detach().unsqueeze(dim=0).to(device), max_new_tokens=output_size(), **gen_kwargs).squeeze()[-output_size():]
active_output = active_model.generate(query_tensor.clone().detach().unsqueeze(dim=0).to(device), max_new_tokens=output_size(), **gen_kwargs).squeeze()[-output_size():]
response_before = tokenizer.decode(reference_output)
response_after = tokenizer.decode(active_output)
reward_before = [active_output[1]["score"] for active_output in sentiment_model([query + response_before], **sentiment_kwargs)][0]
reward_after = [active_output[1]["score"] for active_output in sentiment_model([query + response_after], **sentiment_kwargs)][0]
print("Query:", query)
print("Response before:", response_before)
print("Response after:", response_after)
print("Reward before:", reward_before)
print("Reward after:", reward_after)

Query: This movie is
Response before: This movie is so derivative, twisted,
Response after:  is filled with endless visuals wherein Doris Williams's face,
Reward before: -2.279261350631714
Reward after: -1.653165340423584
